In [ ]:
'''
First, construct Alphabet of all languages
'''
import math as m
NEW_TRAIN_X = open("data/train_set_x_cleaned_nothing.txt","r")
NEW_TRAIN_Y = open("data/train_set_y.txt","r")
D = len(NEW_TRAIN_Y.readlines())
NEW_TRAIN_Y.close()
NEW_TRAIN_Y = open("data/train_set_y.txt","r")
#alphabet for all languages
Alphabet = []
for lineY in NEW_TRAIN_Y:
    lang = (lineY.split(","))[1].replace("\n", "")
    text = (NEW_TRAIN_X.readline()).split(",")[1].replace("\n", "")
    chars = text.split()
    for char in chars:
        if char not in Alphabet:
            Alphabet.append(char)
NEW_TRAIN_X.close()
NEW_TRAIN_Y.close()

from IPython.display import Image
Image("img/TF-IDF.png")
![title](img/TF-IDF.png)
Sourced cited from wikipedia

In [ ]:
'''
Second, construct the neighbourhood of vectors 
each vector contains the TF_IDF value of each unique letter in a sentence
See the above formula
'''
NEW_TRAIN_X = open("data/train_set_x_cleaned_nothing.txt","r")
NEW_TRAIN_Y = open("data/train_set_y.txt","r")
#construct vector base on tf-idf
neighbourhood = [[0 for i in range(len(Alphabet))] for j in range(D)] 
#also store document frequency
df = [0]*len(Alphabet)
#last element in the vector is used to store which language it is
count = 0
langID = [0]*D
for lineY in NEW_TRAIN_Y:
    lang = (lineY.split(","))[1].replace("\n", "")
    text = (NEW_TRAIN_X.readline()).split(",")[1].replace("\n", "")
    chars = text.split()
    #last element in the vector is used to store which language it is
    langID[count] = int(lang)
    for char in chars:
        index = Alphabet.index(char)
        #counting term frequency
        neighbourhood[count][index] = neighbourhood[count][index] + 1
        #make sure it is counted once per document
        if(neighbourhood[count][index] == 1):
            df[index] = df[index] + 1
    count = count+1
    
for i in range(len(df)):
    df[i] = m.log(abs(D)*1.0/df[i])
for i in range(len(neighbourhood)):
    for j in range(len(Alphabet)):
        neighbourhood[i][j] = (neighbourhood[i][j] * 1.0) * df[j]
NEW_TRAIN_X.close()
NEW_TRAIN_Y.close()
    

In [ ]:
'''
Third, Train KNN classifier from Scikit Learn library
NOTE: this step might cause memory error in some machines due to the large memory location required for the neighbourhood vector
'''
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(neighbourhood,langID) 

In [ ]:
'''
Validate

'''

import warnings
warnings.filterwarnings("ignore")

validation = open("data/validation.csv","r")
success = 0
total = 0
fail = 0
for test in validation:
    if(total%1000 == 0):
        print(total)
    lang = (test.split(","))[0]
    letters = ((test.split(","))[1].replace("\n", "")).split(" ")
    vector = [0]*len(Alphabet)
    for letter in letters:
        try:
            index = Alphabet.index(letter)
        except:
            pass
        vector[index] = vector[index]+1
    for i in range(len(Alphabet)):
        vector[i] = (vector[i]*1.0)*df[i] 
    prediction = neigh.predict(vector)
    if(prediction[0] == int(lang)):
        success = success + 1
        total = total + 1
    else:
        fail = fail + 1
        total = total + 1
print (success*1.0/total)
    
        






In [ ]:
'''
Self written KNN learner, not used in the report due to high computational time

#remove language at end of the vector
import numpy as np
import scipy.spatial.distance as dis
validation = open("../data/validation.csv","r")
#use previously calculated df as idf for this case
success = 0
total = 0
fail = 0
for test in validation:
    lang = (test.split(","))[0]
    letters = ((test.split(","))[1].replace("\n", "")).split(" ")
    vector = [0]*len(Alphabet)
    for letter in letters:
        try:
            index = Alphabet.index(letter)
        except:
            pass
        vector[index] = vector[index]+1
    for i in range(len(Alphabet)):
        vector[i] = (vector[i]*1.0)*df[i]
    mostFit = 10000000
    position = 0
    for i in range(len(neighbourhood)):
        dist = dis.euclidean(vector,neighbourhood[i])
        if(dist < mostFit):
            mostFit = dist
            position = i
    if(int(langID[position]) == int(lang)):
        success = success + 1
        total = total + 1
    else:
        fail = fail + 1
        total = total + 1
        
print(success*1.0/total)
'''
        
    